In [101]:
import pandas as pd
import openpyxl
PATH_TO_EXCEL = "/Users/prashantgaurav/Projects/KaajAi/receipts/Bank-Statement-Template-4-TemplateLab.xlsx"



In [103]:
file = pd.read_excel(PATH_TO_EXCEL).fillna('').to_string(index=False)

In [104]:
file

'                 Unnamed: 0                 Unnamed: 1                    Unnamed: 2             Unnamed: 3           Unnamed: 4                     Unnamed: 5           Unnamed: 6  Unnamed: 7\n                                                                                                            Statement of Account                                                                \n                                                                                                                                                                                                \n                                                                                                                                                                                                \n                                                                                                                                                                                                \n                             

In [111]:
import openai
import os
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))

prompt = """I'm going to give you the contents of a pdf of a bank statement. The text containes transaction records along with other things.
you job is to parse each transaction record and return the details like date, description, amount and type. The type column should contain either 
'credit' or 'debit'. Return these information in a tabular format. Do not return anything else just the table"""

docs = [{"role": "system", "content": prompt + '\n' + file}]
response = client.chat.completions.create(
                model="gpt-4",
                messages=docs
            )

In [112]:
resp = response.choices[0].message.content

In [113]:
print(resp)

| Date       | Description           | Amount | Type  |
|------------|-----------------------|--------|-------|
| 04/28/2024 | Petron - C5 Station   | 223.26 | Debit |
| 04/27/2024 | South Star Drug       | 313.39 | Debit |
| 04/27/2024 | Rosewood Condominum   | 582.96 | Debit |
| 04/26/2024 | Grab                  | 125    | Debit |
| 04/22/2024 | Amazon                | 215    | Debit |
| 04/17/2024 | Alba International    | 656.86 | Debit |
| 12/04/2024 | Adobe Sales           | 246    | Debit |
| 07/04/2024 | St. Luke Medical Center | 571.1  | Debit |
| 04/04/2024 | Hotel Sheraton (Las Vegas) | 965 | Debit |


In [114]:
def parse_response_table(table):
    lines = resp.strip().split("\n")
    table = [line.split('|') for line in lines]
    final_table = []
    for line in table:
        new_line = []
        for ele in line:
            ele = ele.replace('-','')
            if ele :
               new_line.append(ele.strip())
        if not new_line:
            continue
        final_table.append(new_line)
    return final_table
x = parse_response_table(table)

In [115]:
x

[['Date', 'Description', 'Amount', 'Type'],
 ['04/28/2024', 'Petron  C5 Station', '223.26', 'Debit'],
 ['04/27/2024', 'South Star Drug', '313.39', 'Debit'],
 ['04/27/2024', 'Rosewood Condominum', '582.96', 'Debit'],
 ['04/26/2024', 'Grab', '125', 'Debit'],
 ['04/22/2024', 'Amazon', '215', 'Debit'],
 ['04/17/2024', 'Alba International', '656.86', 'Debit'],
 ['12/04/2024', 'Adobe Sales', '246', 'Debit'],
 ['07/04/2024', 'St. Luke Medical Center', '571.1', 'Debit'],
 ['04/04/2024', 'Hotel Sheraton (Las Vegas)', '965', 'Debit']]

In [116]:
df = pd.DataFrame(x[1:],columns=x[0])

In [117]:
df

,Date,Description,Amount,Type
0,04/28/2024,Petron C5 Station,223.26,Debit
1,04/27/2024,South Star Drug,313.39,Debit
2,04/27/2024,Rosewood Condominum,582.96,Debit
3,04/26/2024,Grab,125,Debit
4,04/22/2024,Amazon,215,Debit
5,04/17/2024,Alba International,656.86,Debit
6,12/04/2024,Adobe Sales,246,Debit
7,07/04/2024,St. Luke Medical Center,571.1,Debit
8,04/04/2024,Hotel Sheraton (Las Vegas),965,Debit
